In [1]:
import tokenize

import pandas as pd
import numpy as np
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import re
import time
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from collections import Counter
from IPython.display import clear_output

from scripts.tokenizer import ByteTokenizer

from scripts import BpeTokenizer, Model, Trainer, Collator, MyDataset, generate

# Загружаем данные

In [2]:
df = pd.read_csv("data/dataset.csv")
train_texts = df["text"][:-1024].tolist()
eval_texts = df["text"][-1024:].tolist()

# Инициализируем и обучаем токенизатор

In [3]:
tokenizer = BpeTokenizer()
# tokenizer = ByteTokenizer()

In [4]:
tokenizer.train(train_texts[:2048], max_vocab=2048)

pair=(277, 338), freq=52: 100%|██████████| 1789/1789 [03:51<00:00,  7.73it/s]  


In [5]:
encoded = tokenizer.encode(r" случайный текст")

In [6]:
tokenizer.decode(encoded)

' случайный текст'

# Создаем датасеты и Collator

In [7]:
train_dataset = MyDataset(train_texts, tokenizer, max_length=128)
eval_dataset = MyDataset(eval_texts, tokenizer, max_length=128)
collator = Collator(tokenizer.pad_token_id)

100%|██████████| 1024/1024 [00:24<00:00, 42.40it/s]


# Создаем модель

In [8]:
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

In [9]:
model = Model(
    tokenizer.get_vocab_size(), emb_size=128, hidden_size=256, num_layers=2, dropout=0.1
)

# Создаем Trainer и запускаем обучение

In [10]:
trainer = Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    n_epochs=8,
    train_batch_size=32,
    eval_batch_size=32,
    eval_steps=64,
    collator=collator,
    lr=1e-2,
    ignore_index=tokenizer.pad_token_id,
)

In [11]:
trainer.train()

epoch=0.126953125, loss=4.706699848175049:   2%|▏         | 65/4096 [00:17<1:03:30,  1.06it/s]

epoch=0.125, eval_loss=4.681298717856407


epoch=0.251953125, loss=4.012765884399414:   3%|▎         | 129/4096 [00:33<1:04:13,  1.03it/s]

epoch=0.25, eval_loss=4.008522070944309


epoch=0.376953125, loss=3.7434115409851074:   5%|▍         | 193/4096 [00:48<1:04:17,  1.01it/s]

...

epoch=7.75, eval_loss=2.9790947884321213


epoch=7.876953125, loss=3.0027570724487305:  98%|█████████▊| 4033/4096 [18:33<00:58,  1.07it/s]

epoch=7.875, eval_loss=2.9754804968833923


epoch=8.0, loss=2.9552931785583496: 100%|██████████| 4096/4096 [18:49<00:00,  3.63it/s]        

epoch=8.0, eval_loss=2.9725159406661987


# Оцениваем качество и проверяем жадную и случайную генерацию

In [12]:
trainer.evaluate()

2.9725159406661987

In [13]:
generate(model, tokenizer, temperature=0)

'Козерогам стоит быть внимательнее контакты с друзьями и любимыми людьми. В это время вы можете стать несколько излишнейшим вопросам своих усилий. В это время не стоит подумать о том, что вы не можете сделать внимание на привычную деятельность. В это время не стоит подумать о том, что вы не можете сделать внимание на привычную деятельность. В это время не стоит подумать о том, что вы не можете сделать внимание на привычную деятельность. В это время не стоит подумать о том, что вы не можете сделать внимание на привычную деятельность. В это время не стоит подумать о том, что вы не можете сделать внимание на привычную деятельность. В это время не стоит подумать о том, что вы не можете сделать внимание на привычную деятельность. В это время не стоит подключать красоту или вдохновения, развлечений, развлечений, развлечений, развлечений, развлечений, развлечений, развлечений, рассчитывать на то, что вы не можете сделать внимание на привычную деятельность. В это время не стоит подключать крас

In [21]:
generate(model, tokenizer, temperature=0.25, top_k=10)

'На протяжении всего дня вы можете стать своим образом и различной работой, а также не столько надежными своими обязанностями, так как вам не помешает вам стоит быть более коммуникабельными. Подходящий момент для общения с коллегами с родственниками, общениями, ответственными людьми, если вы отдыхом), что сегодня станет более общенами и общениями, общениями, если вы отказываете отношения с окружающим отношениями, общениями, общениями, общениями, общениями, ответственными людьми, ответственными людьми, ответственными людьми, оптимистичными обязанностями, ответственными отношениями, ответственностью, общениями, общениями, общениями, ответственными людьми, общениями, общениями, общениями, общениями, общениями, общениями, особенно с окружающими и общениями, общениями, общениями, общениями, темпераментами, общениями, общениями, общениями, общениями и общениями, общениями, общениями, общениями, общениями, общениями, общениями, особенно с ними обязанности с окружающими и общениями, общениями,